In [9]:
%load_ext autoreload
%autoreload

import dut_func as dut
import serial
import time
import struct
import numpy as np

from misc import *

# dut.drv.serUSB = 'COM19'
drv = dut.drv

ser = serial.Serial('COM6', 9600, timeout=1)
dut.connect(ser)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
dut.disconnect()

Serial port disconnected.


Test serial communication

In [ ]:
with serial.Serial(serUSB, 9600, timeout=1) as ser:
    ser.write(b'101,2')

In [30]:
drv.disconnect()

Serial port disconnected.


Write port H with value 0b0101

In [10]:
drv.gpio_port_write('H', 0xfff1)

In [18]:
value = drv.gpio_port_read('H')
print(f'{value:016b}')

1101111011001111


In [19]:
drv.gpio_adc_fifo_en_write(0xffff)

In [20]:
value = drv.gpio_adc_read()
print(f'{value:016b}')

0000111111111111


In [21]:
drv.spi_dac_write(0x240e)

In [92]:
data = bytes([0b1101_0001, 0b0000_0010, 0b0000_0000])
drv.spi_serial_write(1, data)

In [4]:
voltages = drv.pic_adc_read()
voltages

array([0.        , 0.03465201, 0.03465201, 0.376337  , 2.54007326])

In [51]:
def dac_set(channel, voltage):
    # print(f'DAC: setting ch={channel} to vol={voltage}')
    '''
    '''
#     if dac_set.is_init == False:
#         dac_init(span = 0b010)
    span=0b010

    portName, pinPos = PIC_PINS['PIC_LDAC']
    drv.gpio_pin_set(portName, pinPos)

    portName, pinPos = PIC_PINS['PIC_TGP']
    drv.gpio_pin_set(portName, pinPos)

    portName, pinPos = PIC_PINS['PIC_CLR']
    drv.gpio_pin_reset(portName, pinPos)
    # time.sleep(1)
    drv.gpio_pin_set(portName, pinPos)
    
    cmd = 0b1110
    # address = 0b0000
    # address = DAC_CH[channel]

    data = span | (cmd << 20)
    drv.spi_dac_write(data)
    # print(f'{data:08x}')

    dac_set.is_init = True
    dac_set.span = span

    vlim_lo, vlim_hi = DAC_SPAN[dac_set.span]
    print(f'DAC initialized to a span from {vlim_lo} V to {vlim_hi} V')
    
    

    cmd = 0b0011
    # address = 0b0000
    address = DAC_CH[channel]
    print(address)

    # data = int(voltage)
#     vlim_lo, vlim_hi = 0, 5
    data = int( (voltage - vlim_lo) / (vlim_hi-vlim_lo) * 0xffff  )
    # print(f'{data:08x}')
    data |= (address << 16) | (cmd << 20)
    print(f'{data:032b}')

    drv.spi_dac_write(data)

In [66]:
dac_set('DAC_SCHOTTKY', 5)

DAC initialized to a span from -5 V to 5 V
15
00000000001111111111111111111111


In [69]:
drv.gpio_pin_set(*PIC_PINS['PIC_SHORT_VREF_HI_CMP'])

In [68]:
drv.gpio_pin_reset(*PIC_PINS['PIC_SHORT_VREF_HI_CMP'])

In [96]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [99]:
drv.gpio_pin_reset(*PIC_PINS['WRITE_FWD'])

In [14]:
drv.clk_start('CK_ARRAY')

In [15]:
drv.clk_start('ADC_CK')

In [12]:
drv.clk_stop('ADC_CK')

In [13]:
drv.clk_stop('CK_ARRAY')